# CNN Lesion Test

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

import sys
sys.path.append('..')

from IPython import display
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import warnings
from src.utils import suppress, all_logging_disabled
from src.lesion import (perform_lesion_experiment, do_lesion_hypo_tests,
                        plot_all_damaged_clusters, compute_damaged_cluster_stats,
                        plot_overall_damaged_clusters)
from src.pointers import DATA_PATHS
from src.experiment_tagging import get_model_path

sns.set(style='whitegrid')

Using TensorFlow backend.


In [2]:
MODELS = [('mnist', 'CNN-MNIST'),
          ('cifar10', 'CNN-CIFAR10'),
          ('fashion', 'CNN-FASHION'),
          ('mnist', 'CNN-MNIST+DROPOUT'),
          ('cifar10', 'CNN-CIFAR10+DROPOUT'),
          ('fashion', 'CNN-FASHION+DROPOUT'),
          ('mnist', 'CNN-MNIST+L1REG'),
          ('cifar10', 'CNN-CIFAR10+L1REG'),
          ('fashion', 'CNN-FASHION+L1REG')]

n_clust = 12
n_shuffles = 20
n_workers = 10

In [3]:
%%time

with warnings.catch_warnings():
    warnings.simplefilter('ignore')

    for dataset_name, model_name in tqdm(MODELS):

        (true_results,
         all_random_results,
         metadata,
         evaluation) = perform_lesion_experiment('.' + DATA_PATHS[dataset_name],
                                                 get_model_path(model_name), n_clusters=n_clust,
                                                 n_shuffles=n_shuffles, unpruned=True)

        plot_all_damaged_clusters(true_results, all_random_results, metadata, evaluation,
                                  title=f'{model_name}')

        plot_overall_damaged_clusters(true_results, all_random_results, metadata, evaluation,
                                      title=f'{model_name}')

        # df = compute_damaged_cluster_stats(true_results, all_random_results, metadata, evaluation)
        # display.display(df)

        hypo_results = do_lesion_hypo_tests(evaluation, true_results, all_random_results)
        mean_percentiles = hypo_results['mean_percentiles']
        range_percentiles = hypo_results['range_percentiles']
        chi2_p_means = hypo_results['chi2_p_means']
        chi2_p_ranges = hypo_results['chi2_p_ranges']
        combined_p_means = hypo_results['combined_p_means']
        combined_p_ranges = hypo_results['combined_p_ranges']
        print(f'{model_name}')
        print(f'Mean percentiles: {mean_percentiles}')
        print(f'Range percentiles: {range_percentiles}')
        print(f'chi2 mean p: {chi2_p_means}')
        print(f'chi2 range p: {chi2_p_means}')
        print(f'combined mean p: {combined_p_means}')
        print(f'combined range p: {combined_p_ranges}')
        print()

  0%|          | 0/9 [00:00<?, ?it/s]